In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()


True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH05-Memory")


LangSmith 추적을 시작합니다.
[프로젝트명]
CH05-Memory


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# 단계 1: 문서 로드(Load Documents)
from langchain.document_loaders import MongodbLoader
from server.db import DB
from server.db import get_mongo_connection_string

channel_id = 1890652954

loader = MongodbLoader(
    connection_string=get_mongo_connection_string(),
    db_name=DB.NAME,
    collection_name=DB.COLLECTION.CHANNEL.DATA,
    filter_criteria={"channelId": channel_id}, # 데이터베이스에서 조회할 기준 (쿼리)
    field_names=("text",),
    metadata_names=("id", "timestamp", "channelId", "views", "url"), # 메타데이터로 지정할 필드 목록
)

import nest_asyncio
nest_asyncio.apply()  # Jupyter Notebook에서 asyncio 실행 문제 해결

docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
splitted_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embedding = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성한다.
vectorstore = FAISS.from_documents(documents=splitted_documents, embedding=embedding)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성한다.
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

In [16]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# 프롬프트 정의
prompt = PromptTemplate.from_template(
    """You are an assistant responding to inquiries from an investigator trying to investigate a drug-selling channel. 
Below is chat data collected from a Telegram channel where drugs are being sold.
Based on this chat data, answer questions about the transaction details of the channel.
If you don't know the answer, just say that you don't know.
Answer in Korean.

#Previous Chat History:
{chat_history}

#Question: 
{question} 

#Context: 
{context} 

#Answer:""")

# llm 생성
llm = ChatOpenAI(model_name="gpt-4o")

# 일반 Chain 생성
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [17]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

chain_with_history.invoke(
    # 질문 입력
    {"question": "이 채널에서 판매되는 마약의 지역을 말해줘."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'이 채널에서 판매되는 마약의 종류는 인디카 떨, 아프간 쿠쉬, 멕시코산 퓨어 원석 케타민, 아이스, 떨액, S케이, 브액입니다.'

In [19]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "내가 이전에 물어본 내용과 그 답변이 뭐였지?", "context": retriever},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'당신이 이전에 물어본 내용과 그 답변은 다음과 같습니다.\n\n1. 질문: 이 채널에서 판매되는 마약의 종류를 말해줘.\n   답변: 이 채널에서 판매되는 마약의 종류는 인디카 떨, 아프간 쿠쉬, 멕시코산 퓨어 원석 케타민, 아이스, 떨액, S케이, 브액입니다.\n\n2. 질문: 이 채널에서 판매되는 마약의 지역을 말해줘.\n   답변: 이 채널에서 판매되는 마약의 지역은 부산, 창원, 울산, 서울을 비롯한 수도권, 광주, 천안 등입니다.'

In [ ]:
# TODO: https://wikidocs.net/233813 를 참고해서 SQLite에 대화내역 오프라인 저장하고 불러오기